In [4]:
import time
import datetime as dt
import numpy as np
import pyvisa

In [5]:
# rm = visa.ResourceManager('C:\\Windows\\System32\\visa32.dll')

#save_path = r"C:/Users/IPMU\Documents/Python Scripts/devinit/"
save_path = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"

# look up instruments
rm = pyvisa.ResourceManager()
print("resource manager : ", rm)
rl = rm.list_resources()
print("List of resources: \n", rl)

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')


In [6]:
for i,name in enumerate(rl):
    if "GPIB" in name:
        my_instrument = rm.open_resource(name)
        print("Resource[",i,"]:", name, "\n", my_instrument.query("*IDN?"))

Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40



In [7]:
KT2700 = [rm.open_resource(rl[3])]
filename = save_path + dt.datetime.now().strftime("%Y%m%d%H%M%S") + "_MULTIMETER_KT2700.csv"
print(filename)

columnname = ['abstime','reltime','Voltage[V]']
columnname = ['reltime','Voltage[V]']

try:
    with open(filename, mode="a") as f:
        print(*columnname, sep=", ", file=f)
except:
    pass

REFRESHRATE = 0.1  # [sec] for frequency of daq

C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221110613_MULTIMETER_KT2700.csv


In [8]:
KT2700[0].write('*RST')
KT2700[0].write(':INITiate:CONTinuous OFF;:ABORt')
KT2700[0].write(':SENSe:VOLTage:DC:RANGe:AUTO ON')
KT2700[0].write(':SENSe:FUNCtion "VOLTage:DC"')
KT2700[0].write(':SENSe:VOLTage:DC:NPLC 0.01') # Use lowest NPLC setting for fastest readings.
KT2700[0].write(':SYSTem:AZERo:STATe OFF') #  Disable autozero to increase speed, but may cause drift over time.
KT2700[0].write(':SENSe:VOLTage:DC:AVERage:STATe OFF') # Turn off filter for speed.
KT2700[0].write(':TRIGger:COUNt 1')
KT2700[0].write('READ?')
buf_str = KT2700[0].read()


In [5]:
def getVotage(self):
    self.write('READ?')
    buf_str = self.read()
    buf_str = buf_str.strip()  # 改行文字を削除
    buf_str_list = buf_str.split(',')  # カンマで分割
    buf_float = float(buf_str_list[0][:-3]) # [:-3] is coresponding to "VDC"
    return buf_float

In [10]:
StarTime = time.perf_counter()

while True:

    #CurrentAbsTime = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    CurrentRerTime = time.perf_counter() - StarTime
    #print(time.perf_counter())
    #arr = [CurrentAbsTime, CurrentRerTime]

    arr = [CurrentRerTime]
    
    buf = getVotage(KT2700[0])
    arr.append(buf)

    print(arr)
    time.sleep(REFRESHRATE)

    try:
        with open(filename, mode="a") as f:
            print(*arr, sep=", ", file=f)

    except FileNotFoundError:

        print("FileNotFoundError... :'-(")
        filename = save_path + dt.datetime.now().strftime("%Y%m%d%H%M%S") + ".csv"
        with open(filename, mode="a") as f:
            print(*columnname, sep=", ", file=f)
            print(*arr, sep=", ", file=f)

[8.629999999953952e-05, 0.0190714579]
[0.1507628999999966, -0.0139528811]
[0.28528549999999697, 0.0138909761]
[0.427593599999998, 0.00435504876]
[0.5622969999999974, 0.00687035918]
[0.7063502999999969, 0.002609357]
[0.8379707999999972, -0.0121051138]
[0.9746555999999984, -0.0603007451]
[1.1052378999999988, -0.0132119386]
[1.2488310999999968, -0.0305196978]
[1.3839687999999981, -0.0247137416]
[1.5257359999999984, -0.0297776088]
[1.668893899999997, -0.0620865747]
[1.8069390999999975, -0.0404432714]
[1.9412952999999966, -0.0476714708]
[2.090041499999998, -0.0516766869]
[2.2371269, -0.0531723313]
[2.385258499999999, -0.0440642536]
[2.5236326999999967, -0.0508462787]
[2.6602917999999995, -0.0587638058]
[2.7937908999999976, -0.0442596897]
[2.9454284999999985, -0.0240829103]
[3.087308099999998, -0.0168455355]
[3.2207674999999973, -0.0251289457]
[3.3685674999999975, -0.0364690274]
[3.5108578999999978, -0.0704525635]
[3.644529200000001, -0.0406130254]
[3.7818427999999997, -0.0476313271]
[3.9177

KeyboardInterrupt: 

In [2]:
import lib_KT2700

In [4]:
rm,rl = lib_KT2700.query()

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')
Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40



In [5]:
save_path = save_path = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"
columnnames = ['reltime','Voltage[V]']

filename = lib_KT2700.create_csv(save_path,columnnames)
KT2700 = rm.open_resource(rl[3])

Generated filename: C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221111056_MULTIMETER_KT2700.csv


In [7]:
lib_KT2700.initialize(instrument=KT2700)

-9.34909936E-03VDC,+329847.005SECS,+15290280RDNG#

KT2700 initialization complete.


In [8]:
lib_KT2700.DataAcquisition(filename,save_path,columnnames,refresh_rate=0.01,instrument=KT2700)

[9.999999406318238e-08, -0.0153690632]
[0.0669806999999949, -0.00765385246]
[0.11639959999999405, -0.0191466659]
[0.16201209999999833, -0.0141916806]
[0.2069557999999958, -0.0140953343]
[0.2496858999999958, -0.0163009912]
[0.2952668000000003, -0.0257946067]
[0.3475502000000006, -0.0204737317]
[0.39493359999999456, -0.0271836072]
[0.44508739999999847, -0.0242438782]
[0.4954442999999955, -0.0327579677]
[0.5458605999999975, -0.027993381]
[0.6017043999999956, -0.0329128094]
[0.6477223999999993, -0.0320422463]
[0.7024822999999998, -0.0378425606]
[0.7531807999999955, -0.0344130695]
[0.8056173999999956, -0.0368882678]
[0.8623826999999977, -0.0434192158]
[0.9122666999999964, -0.0394701362]
[0.9622335999999976, -0.0466330759]
[1.0128580999999954, -0.0441062599]
[1.0620064999999954, -0.052514825]
[1.1116904000000005, -0.0489832498]
[1.1621095999999937, -0.0564386696]
[1.2124250999999973, -0.0534610897]
[1.2618182000000004, -0.0632139221]
[1.3144615999999942, -0.0601686724]
[1.3619546999999983, -

KeyboardInterrupt: 

In [6]:
import lib_KT2700
rm,rl = lib_KT2700.query()

save_path = save_path = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"
columnnames = ['reltime','Voltage[V]']

filename = lib_KT2700.create_csv(save_path,columnnames)
KT2700 = rm.open_resource(rl[3])

lib_KT2700.initialize(instrument=KT2700)

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')
Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40

Generated filename: C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221182413_MULTIMETER_KT2700.csv
-9.53769777E-03VDC,+12463.945SECS,+04117RDNG#

KT2700 initialization complete.


In [8]:
import time
import numpy as np
import nidaqmx
from nidaqmx.constants import AcquisitionType
from nidaqmx.errors import DaqError
import datetime as dt
from itertools import islice
def time_provider(sampling_rate=512):
    """
    Generator that yields continuous relative time values (in seconds)
    at the specified sampling rate.
    Parameters:
        sampling_rate (int): Number of samples per second.
    Yields:
        float: The relative time (in seconds) since the generator started.
    """
    interval = 1.0 / sampling_rate  # Time interval between samples in seconds
    start_time = time.perf_counter()  # Record the start time using a high-resolution timer
    next_sample_time = start_time
    while True:
        current_time = time.perf_counter()
        # Wait until the next scheduled sample time is reached
        if current_time < next_sample_time:
            time.sleep(next_sample_time - current_time)
        # Yield the elapsed time (relative to start)
        yield next_sample_time - start_time
        # Update the next sample time for the next iteration
        next_sample_time += interval

def DataAcquisition(tp, save_path, filename, columnname, sampling_rate=512):
    """
    Continuously reads voltage data from NI-DAQ channels and appends the data to a CSV file.
    The function performs the following tasks:
      1. Sets up the NI-DAQ task to read voltage data from four channels:
         - "cDAQ1Mod2/ai0": Current Phase A
         - "cDAQ1Mod2/ai1": Current Phase B
         - "cDAQ1Mod1/ai0": Voltage Phase A
         - "cDAQ1Mod1/ai2": Voltage Phase B
      2. Configures the sampling clock for continuous acquisition.
      3. Reads a block of data (number of samples per channel equals sampling_rate).
      4. Creates a time vector for the current block using the provided time provider.
      5. Writes the data (with time information) to a CSV file.
         If the file is not found, it creates a new CSV file with a header.
    Parameters:
        tp (generator): A time provider generator yielding relative time values.
        save_path (str): Directory where the CSV file will be saved.
        filename (str): The initial file path for saving data.
        columnname (list): List of column names for the CSV header.
        sampling_rate (int): Number of samples to acquire per second.
    """
    while True:
        # Try to read data from NI-DAQ channels
        try:
            # Create a new NI-DAQ task for each block of acquisition
            with nidaqmx.Task() as task:
                # Add analog input channels for current and voltage measurements
                task.ai_channels.add_ai_voltage_chan("cDAQ1Mod2/ai0")  # Current Phase A
                task.ai_channels.add_ai_voltage_chan("cDAQ1Mod2/ai1")  # Current Phase B
                task.ai_channels.add_ai_voltage_chan("cDAQ1Mod1/ai0")  # Voltage Phase A
                task.ai_channels.add_ai_voltage_chan("cDAQ1Mod1/ai2")  # Voltage Phase B
                # Configure the sampling clock for continuous acquisition
                task.timing.cfg_samp_clk_timing(
                    rate=sampling_rate,
                    sample_mode=AcquisitionType.CONTINUOUS,
                    samps_per_chan=int(sampling_rate),
                )
                # Read a block of data (number of samples per channel equals sampling_rate)
                data = np.array(task.read(number_of_samples_per_channel=sampling_rate))
        except DaqError as e:
            print(f"Reading Error: {e}")
            continue  # Skip this iteration if a reading error occurs
        # Generate a time vector for the current block by collecting sampling_rate time points from tp
        #timedata = np.array([next(tp) for _ in range(sampling_rate)])
        timedata = np.array(list(islice(tp,sampling_rate)))
        # Stack the time data and the acquired data vertically.
        # Assumes that the data shape aligns with timedata, e.g., one row per channel.
        arr = np.vstack([timedata, data])
        # Try to append the data block to the CSV file
        try:
            with open(filename, mode="a") as f:
                np.savetxt(f, arr.T, delimiter=",")
        except FileNotFoundError:
            # If the file is not found, create a new file with a header
            print("FileNotFoundError occurred... Creating a new file.")
            filename = save_path + dt.datetime.now().strftime("%Y%m%d%H%M%S") + ".csv"
            with open(filename, mode="a") as f:
                # Write the header line then the data block
                print(*columnname, sep=", ", file=f)
                np.savetxt(f, arr.T, delimiter=",")
if __name__ == '__main__':
    # Create a time provider generator instance
    SAMPRINGRATE = 1
    tp = time_provider(sampling_rate=SAMPRINGRATE)
    # Set parameters for the data acquisition
    #save_path = "./data/"
    #filename = "./data/initial_file.csv"
    #columnname = ["Time (s)", "Current Phase A", "Current Phase B", "Voltage Phase A", "Voltage Phase B"]
    # Start continuous data acquisition, passing the time provider as an argument
    DataAcquisition(tp, save_path, filename, columnnames, sampling_rate=SAMPRINGRATE)

KeyboardInterrupt: 